In [2]:
%matplotlib inline
%pylab inline
%config InlineBackend.figure_formats = ['retina']

# Connect to MongoDB
from pymongo import MongoClient

# Basics + EDA
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set(context='notebook', style='whitegrid', font_scale=1.2)


# plt.rcParams["figure.figsize"] = (8, 4)
# plt.rcParams["xtick.labelsize"] = 8

Populating the interactive namespace from numpy and matplotlib


In this notebook, we will: 
- a. Get the data from MongoDB and put it into a dataframe. 
- b. Do data cleaning: 
        1. Clean up null values: drop columns with all null values & fill na's with appropriate values. 
        2. Analyze data types and change according to what we will be using downstream. We will also extract information from the dictionaries if needed. Lastly we will identify the columns with repeat same values (eg. since this is one collection by one artist, the creator & collection columns have the same 1 value throughout) and not carry them downstream. 
        3. Save the processed data into a csv to be used downstream. 

## Get Data from MongoDB and Initial Look

In [3]:
client = MongoClient()

In [4]:
client.list_database_names()

['admin', 'books', 'config', 'events', 'local', 'outings', 'pak']

In [91]:
db = client.pak
lostPoets = db.lostPoets

In [92]:
cursor = db.lostPoets.find({})
len(list(cursor))

10020

In [93]:
lostPoets_data = lostPoets.find()
poets = pd.DataFrame(lostPoets_data)
poets.head(3)

,_id,id,token_id,num_sales,background_color,image_url,image_preview_url,image_thumbnail_url,image_original_url,animation_url,...,owner,sell_orders,creator,traits,last_sale,top_bid,listing_date,is_presale,transfer_fee_payment_token,transfer_fee
0,61b6bdd216629c8ab9d260d4,59080697,1,0,None,https://lh3.googleusercontent.com/ulT6Iu_7798l...,https://lh3.googleusercontent.com/ulT6Iu_7798l...,https://lh3.googleusercontent.com/ulT6Iu_7798l...,https://d1xxei964ioe0z.cloudfront.net/full/a16...,None,...,"{'user': {'username': 'pak'}, 'profile_img_url...",None,"{'user': {'username': 'pak'}, 'profile_img_url...","[{'trait_type': 'Named', 'value': 'True', 'dis...",None,None,None,False,None,None
1,61b6bdd216629c8ab9d260d5,59350999,2,0,None,https://lh3.googleusercontent.com/gtvCMMER8xEL...,https://lh3.googleusercontent.com/gtvCMMER8xEL...,https://lh3.googleusercontent.com/gtvCMMER8xEL...,https://d1xxei964ioe0z.cloudfront.net/full/622...,None,...,"{'user': {'username': None}, 'profile_img_url'...",None,"{'user': {'username': 'pak'}, 'profile_img_url...","[{'trait_type': 'Origin', 'value': '7THK', 'di...",None,None,None,False,None,None
2,61b6bdd216629c8ab9d260d6,59351004,3,0,None,https://lh3.googleusercontent.com/xgxBi9sL8ZOT...,https://lh3.googleusercontent.com/xgxBi9sL8ZOT...,https://lh3.googleusercontent.com/xgxBi9sL8ZOT...,https://d1xxei964ioe0z.cloudfront.net/full/304...,None,...,{'user': {'username': 'LordTruffington_Vault'}...,None,"{'user': {'username': 'pak'}, 'profile_img_url...","[{'trait_type': 'Origin', 'value': '6AF2', 'di...",None,None,None,False,None,None


**Initial look**

In [94]:
poets.columns

Index(['_id', 'id', 'token_id', 'num_sales', 'background_color', 'image_url',
       'image_preview_url', 'image_thumbnail_url', 'image_original_url',
       'animation_url', 'animation_original_url', 'name', 'description',
       'external_link', 'asset_contract', 'permalink', 'collection',
       'decimals', 'token_metadata', 'owner', 'sell_orders', 'creator',
       'traits', 'last_sale', 'top_bid', 'listing_date', 'is_presale',
       'transfer_fee_payment_token', 'transfer_fee'],
      dtype='object')

In [95]:
poets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10020 entries, 0 to 10019
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   _id                         10020 non-null  object
 1   id                          10020 non-null  int64 
 2   token_id                    10020 non-null  object
 3   num_sales                   10020 non-null  int64 
 4   background_color            0 non-null      object
 5   image_url                   10020 non-null  object
 6   image_preview_url           10020 non-null  object
 7   image_thumbnail_url         10020 non-null  object
 8   image_original_url          10020 non-null  object
 9   animation_url               0 non-null      object
 10  animation_original_url      0 non-null      object
 11  name                        10020 non-null  object
 12  description                 829 non-null    object
 13  external_link               0 non-null      ob

## Data Cleaning

### 1. Null values

In [96]:
poets.dropna(axis = 1, how='all', inplace = True)

In [97]:
poets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10020 entries, 0 to 10019
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   _id                  10020 non-null  object
 1   id                   10020 non-null  int64 
 2   token_id             10020 non-null  object
 3   num_sales            10020 non-null  int64 
 4   image_url            10020 non-null  object
 5   image_preview_url    10020 non-null  object
 6   image_thumbnail_url  10020 non-null  object
 7   image_original_url   10020 non-null  object
 8   name                 10020 non-null  object
 9   description          829 non-null    object
 10  asset_contract       10020 non-null  object
 11  permalink            10020 non-null  object
 12  collection           10020 non-null  object
 13  decimals             10020 non-null  int64 
 14  token_metadata       10020 non-null  object
 15  owner                10020 non-null  object
 16  sell

**description** 

In [98]:
poets.description[:5]

0    None
1    None
2    None
3    None
4    None
Name: description, dtype: object

In [99]:
poets[poets.description.notnull() == True].head(3)

,_id,id,token_id,num_sales,image_url,image_preview_url,image_thumbnail_url,image_original_url,name,description,...,permalink,collection,decimals,token_metadata,owner,sell_orders,creator,traits,last_sale,is_presale
18,61b6bdd216629c8ab9d260e6,59351062,19,0,https://lh3.googleusercontent.com/9jyoC4n8xE-d...,https://lh3.googleusercontent.com/9jyoC4n8xE-d...,https://lh3.googleusercontent.com/9jyoC4n8xE-d...,https://d1xxei964ioe0z.cloudfront.net/full/b33...,ROBERTFROST,the road not taken\n\nby robert frost\n\ntwo r...,...,https://opensea.io/assets/0x4b3406a41399c7fd2b...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://lostpoets.api.manifoldxyz.dev/metadata/19,"{'user': {'username': 'billsfan86'}, 'profile_...",None,"{'user': {'username': 'pak'}, 'profile_img_url...","[{'trait_type': 'Class', 'value': 'Origin', 'd...",None,False
30,61b6bdd216629c8ab9d260f2,59351097,31,1,https://lh3.googleusercontent.com/Tj44Eh9grNW-...,https://lh3.googleusercontent.com/Tj44Eh9grNW-...,https://lh3.googleusercontent.com/Tj44Eh9grNW-...,https://d1xxei964ioe0z.cloudfront.net/full/1e7...,Origin #8WI4,origin born into happiness\n\npatiently waitin...,...,https://opensea.io/assets/0x4b3406a41399c7fd2b...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://lostpoets.api.manifoldxyz.dev/metadata/31,"{'user': {'username': 'GAMT'}, 'profile_img_ur...",None,"{'user': {'username': 'pak'}, 'profile_img_url...","[{'trait_type': 'Word', 'value': 'can', 'displ...","{'asset': {'token_id': '31', 'decimals': 0}, '...",False
48,61b6bdd316629c8ab9d26104,59351153,49,1,https://lh3.googleusercontent.com/owHio9tnblmG...,https://lh3.googleusercontent.com/owHio9tnblmG...,https://lh3.googleusercontent.com/owHio9tnblmG...,https://d1xxei964ioe0z.cloudfront.net/full/6fc...,TURING,we have lost even this twilight\n\nno one saw ...,...,https://opensea.io/assets/0x4b3406a41399c7fd2b...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://lostpoets.api.manifoldxyz.dev/metadata/49,"{'user': None, 'profile_img_url': 'https://sto...",None,"{'user': {'username': 'pak'}, 'profile_img_url...","[{'trait_type': 'Class', 'value': 'Origin', 'd...","{'asset': {'token_id': '49', 'decimals': 0}, '...",False


In [101]:
poets.description.iloc[18]

'the road not taken\n\nby robert frost\n\ntwo roads diverged in a yellow wood\n\nand sorry i could not travel both\n\nand be one traveler long i stood\n\nand looked down one as far as i could\n\nto where it bent in the undergrowth\n\nthen took the other as just as fair\n\nand having perhaps the better claim\n\nbecause it was grassy and wanted wear\n\nthough as for that the passing there\n\nhad worn them really about the same\n\nand both that morning equally lay\n\nin leaves no step had trodden black\n\noh i kept the first for another day\n\nyet knowing how way leads on to way\n\ni doubted if i should ever come back\n\ni shall be telling this with a sigh\n\nsomewhere ages and ages hence\n\ntwo roads diverged in a wood and i\n\ni took the one less traveled by\n\nand that has made all the difference'

Good data for NLP purposes. Not this project but great to have let's keep it for now. 

In [102]:
poets.description = poets.description.fillna(value='No Description Yet')

**sell_orders** 

In [103]:
poets.sell_orders[:5]

0    None
1    None
2    None
3    None
4    None
Name: sell_orders, dtype: object

In [104]:
poets[poets.sell_orders.notnull() == True].head(3)

,_id,id,token_id,num_sales,image_url,image_preview_url,image_thumbnail_url,image_original_url,name,description,...,permalink,collection,decimals,token_metadata,owner,sell_orders,creator,traits,last_sale,is_presale
11,61b6bdd216629c8ab9d260df,59351041,12,0,https://lh3.googleusercontent.com/2Eurj_xFsb12...,https://lh3.googleusercontent.com/2Eurj_xFsb12...,https://lh3.googleusercontent.com/2Eurj_xFsb12...,https://d1xxei964ioe0z.cloudfront.net/full/6ac...,LEGEND,No Description Yet,...,https://opensea.io/assets/0x4b3406a41399c7fd2b...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://lostpoets.api.manifoldxyz.dev/metadata/12,"{'user': {'username': 'snow_glacier'}, 'profil...",[{'created_date': '2021-09-25T19:17:23.314185'...,"{'user': {'username': 'pak'}, 'profile_img_url...","[{'trait_type': 'Class', 'value': 'Origin', 'd...",None,False
17,61b6bdd216629c8ab9d260e5,59351060,18,0,https://lh3.googleusercontent.com/LXrXttMs0rP8...,https://lh3.googleusercontent.com/LXrXttMs0rP8...,https://lh3.googleusercontent.com/LXrXttMs0rP8...,https://d1xxei964ioe0z.cloudfront.net/full/a32...,Origin #I5D1,No Description Yet,...,https://opensea.io/assets/0x4b3406a41399c7fd2b...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://lostpoets.api.manifoldxyz.dev/metadata/18,"{'user': {'username': 'ultraparadise'}, 'profi...",[{'created_date': '2021-10-09T19:48:12.127717'...,"{'user': {'username': 'pak'}, 'profile_img_url...","[{'trait_type': 'Origin', 'value': 'I5D1', 'di...",None,False
19,61b6bdd216629c8ab9d260e7,59351063,20,0,https://lh3.googleusercontent.com/BaSB-kq94BQ3...,https://lh3.googleusercontent.com/BaSB-kq94BQ3...,https://lh3.googleusercontent.com/BaSB-kq94BQ3...,https://d1xxei964ioe0z.cloudfront.net/full/775...,Origin #VIAK,No Description Yet,...,https://opensea.io/assets/0x4b3406a41399c7fd2b...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://lostpoets.api.manifoldxyz.dev/metadata/20,"{'user': {'username': '333666999'}, 'profile_i...",[{'created_date': '2021-10-01T15:23:31.267293'...,"{'user': {'username': 'pak'}, 'profile_img_url...","[{'trait_type': 'Origin', 'value': 'VIAK', 'di...",None,False


In [105]:
type(poets.sell_orders.iloc[11])

list

In [106]:
poets.sell_orders.iloc[11]

[{'created_date': '2021-09-25T19:17:23.314185',
  'closing_date': None,
  'closing_extendable': False,
  'expiration_time': 0,
  'listing_time': 1632597325,
  'order_hash': '0x27e5cec922e350cc73aa221929c49d5d7cdcfcbe9f903130363b7c88fe18b5c7',
  'metadata': {'asset': {'id': '12',
    'address': '0x4b3406a41399c7fd2ba65cbc93697ad9e7ea61e5'},
   'schema': 'ERC721'},
  'exchange': '0x7be8076f4ea4a4ad08075c2508e481d6c946d12b',
  'maker': {'user': 60074,
   'profile_img_url': 'https://storage.googleapis.com/opensea-static/opensea-profile/12.png',
   'address': '0xa362a9676db12606d339d10433c1af3618bca1d4',
   'config': ''},
  'taker': {'user': 1766,
   'profile_img_url': 'https://storage.googleapis.com/opensea-static/opensea-profile/1.png',
   'address': '0x0000000000000000000000000000000000000000',
   'config': ''},
  'current_price': '3690000000000000000000',
  'current_bounty': '36900000000000000000',
  'bounty_multiple': '0.01',
  'maker_relayer_fee': '1250',
  'taker_relayer_fee': '0',
 

In [108]:
poets.sell_orders = poets.sell_orders.fillna(value={})

**creator** 

In [109]:
poets.creator[:5]

0    {'user': {'username': 'pak'}, 'profile_img_url...
1    {'user': {'username': 'pak'}, 'profile_img_url...
2    {'user': {'username': 'pak'}, 'profile_img_url...
3    {'user': {'username': 'pak'}, 'profile_img_url...
4    {'user': {'username': 'pak'}, 'profile_img_url...
Name: creator, dtype: object

In [113]:
poets[poets.creator.isnull() == True].head(2)

,_id,id,token_id,num_sales,image_url,image_preview_url,image_thumbnail_url,image_original_url,name,description,...,permalink,collection,decimals,token_metadata,owner,sell_orders,creator,traits,last_sale,is_presale
4579,61b6be3616629c8ab9d272b7,59283406,5198,0,https://lh3.googleusercontent.com/8diNIW7nLWYM...,https://lh3.googleusercontent.com/8diNIW7nLWYM...,https://lh3.googleusercontent.com/8diNIW7nLWYM...,https://d1xxei964ioe0z.cloudfront.net/full/2a9...,Sky,as above so below,...,https://opensea.io/assets/0x4b3406a41399c7fd2b...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://lostpoets.api.manifoldxyz.dev/metadata...,"{'user': {'username': 'devboi'}, 'profile_img_...",NaN,None,"[{'trait_type': 'Class', 'value': 'Poet', 'dis...",None,False
4580,61b6be3616629c8ab9d272b8,59283408,5199,0,https://lh3.googleusercontent.com/_uPGSYx4D3V6...,https://lh3.googleusercontent.com/_uPGSYx4D3V6...,https://lh3.googleusercontent.com/_uPGSYx4D3V6...,https://d1xxei964ioe0z.cloudfront.net/full/716...,Poet #5199,No Description Yet,...,https://opensea.io/assets/0x4b3406a41399c7fd2b...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://lostpoets.api.manifoldxyz.dev/metadata...,"{'user': {'username': 'kilrizzy'}, 'profile_im...",[{'created_date': '2021-12-10T18:29:58.884449'...,None,"[{'trait_type': 'Genre', 'value': 'ioh', 'disp...",None,False


In [115]:
type(poets.creator.iloc[0])

dict

In [117]:
poets.creator.iloc[0]

{'user': {'username': 'pak'},
 'profile_img_url': 'https://storage.googleapis.com/opensea-static/opensea-profile/23.png',
 'address': '0x2ce780d7c743a57791b835a9d6f998b15bbba5a4',
 'config': 'verified'}

Since this is a one artist project, this column is not going to help us much but let's assign the same value to each column.

In [118]:
poets.creator = poets.sell_orders.fillna(value=poets.creator.iloc[0])

**last_sale** 

In [119]:
poets.last_sale[:5]

0                                                 None
1                                                 None
2                                                 None
3    {'asset': {'token_id': '4', 'decimals': 0}, 'a...
4                                                 None
Name: last_sale, dtype: object

In [120]:
poets[poets.last_sale.notnull() == True].head(3)

,_id,id,token_id,num_sales,image_url,image_preview_url,image_thumbnail_url,image_original_url,name,description,...,permalink,collection,decimals,token_metadata,owner,sell_orders,creator,traits,last_sale,is_presale
3,61b6bdd216629c8ab9d260d7,59351008,4,1,https://lh3.googleusercontent.com/KHWZ4GF4GOIU...,https://lh3.googleusercontent.com/KHWZ4GF4GOIU...,https://lh3.googleusercontent.com/KHWZ4GF4GOIU...,https://d1xxei964ioe0z.cloudfront.net/full/073...,BOND,No Description Yet,...,https://opensea.io/assets/0x4b3406a41399c7fd2b...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://lostpoets.api.manifoldxyz.dev/metadata/4,"{'user': {'username': 'TopHand'}, 'profile_img...",NaN,NaN,"[{'trait_type': 'Class', 'value': 'Origin', 'd...","{'asset': {'token_id': '4', 'decimals': 0}, 'a...",False
30,61b6bdd216629c8ab9d260f2,59351097,31,1,https://lh3.googleusercontent.com/Tj44Eh9grNW-...,https://lh3.googleusercontent.com/Tj44Eh9grNW-...,https://lh3.googleusercontent.com/Tj44Eh9grNW-...,https://d1xxei964ioe0z.cloudfront.net/full/1e7...,Origin #8WI4,origin born into happiness\n\npatiently waitin...,...,https://opensea.io/assets/0x4b3406a41399c7fd2b...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://lostpoets.api.manifoldxyz.dev/metadata/31,"{'user': {'username': 'GAMT'}, 'profile_img_ur...",NaN,NaN,"[{'trait_type': 'Word', 'value': 'can', 'displ...","{'asset': {'token_id': '31', 'decimals': 0}, '...",False
37,61b6bdd216629c8ab9d260f9,59351122,38,1,https://lh3.googleusercontent.com/SSiWWa0qau1V...,https://lh3.googleusercontent.com/SSiWWa0qau1V...,https://lh3.googleusercontent.com/SSiWWa0qau1V...,https://d1xxei964ioe0z.cloudfront.net/full/0c9...,Origin #G31Y,No Description Yet,...,https://opensea.io/assets/0x4b3406a41399c7fd2b...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://lostpoets.api.manifoldxyz.dev/metadata/38,"{'user': {'username': 'mexpex'}, 'profile_img_...",NaN,NaN,"[{'trait_type': 'Origin', 'value': 'G31Y', 'di...","{'asset': {'token_id': '38', 'decimals': 0}, '...",False


Let's look at the information in this last_sale column.

In [122]:
type(poets.last_sale.iloc[3])

dict

In [123]:
poets.last_sale.iloc[3]

{'asset': {'token_id': '4', 'decimals': 0},
 'asset_bundle': None,
 'event_type': 'successful',
 'event_timestamp': '2021-09-26T23:27:53',
 'auction_type': None,
 'total_price': '37000000000000000000',
 'payment_token': {'id': 1,
  'symbol': 'ETH',
  'address': '0x0000000000000000000000000000000000000000',
  'image_url': 'https://storage.opensea.io/files/6f8e2979d428180222796ff4a33ab929.svg',
  'name': 'Ether',
  'decimals': 18,
  'eth_price': '1.000000000000000',
  'usd_price': '3979.219999999999800000'},
 'transaction': {'block_hash': '0x277517c12baa4d5889182fa726bad7345a72d4faa9eb7003cdd5d6cb8419fbf1',
  'block_number': '13304385',
  'from_account': {'user': {'username': 'TOPHANDS'},
   'profile_img_url': 'https://storage.googleapis.com/opensea-static/opensea-profile/32.png',
   'address': '0xfed751d3f6f122555c34c1726ff67eacdc500bfa',
   'config': ''},
  'id': 171368158,
  'timestamp': '2021-09-26T23:27:53',
  'to_account': {'user': {'username': 'OpenSea-Orders'},
   'profile_img_ur

If the event type is successful, it shows that this item was at least once sold. But we have our events/ sales analysis to tell us this so essentially this will not be a column that we can use. But let's keep it and assign all the null values an indicator that it hasn't been sold.

In [124]:
poets.last_sale = poets.last_sale.fillna(value= {'asset': {'event_type': 'unsuccessful'}})

### 2. Column Types Analysis & Identifying Repeat-Value Columns

Let's make sure/change the datatypes in columns are the right ones for our use case. Also let's identify columns with repeat values & not enough/ valuable info. 

In [128]:
poets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10020 entries, 0 to 10019
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   _id                  10020 non-null  object
 1   id                   10020 non-null  int64 
 2   token_id             10020 non-null  int64 
 3   num_sales            10020 non-null  int64 
 4   image_url            10020 non-null  object
 5   image_preview_url    10020 non-null  object
 6   image_thumbnail_url  10020 non-null  object
 7   image_original_url   10020 non-null  object
 8   name                 10020 non-null  object
 9   description          10020 non-null  object
 10  asset_contract       10020 non-null  object
 11  permalink            10020 non-null  object
 12  collection           10020 non-null  object
 13  decimals             10020 non-null  int64 
 14  token_metadata       10020 non-null  object
 15  owner                10020 non-null  object
 16  sell

**token_id** 

In [125]:
poets.token_id.nunique()

10020

In [126]:
type(poets.token_id[0])

str

In [127]:
poets.token_id = poets.token_id.astype(int) 

**num_sales** 

In [133]:
poets.num_sales.unique()

array([0, 1, 2, 3, 4])

In [136]:
poets.num_sales[poets.num_sales != 0].head(3)

3     1
30    1
37    1
Name: num_sales, dtype: int64

**URL's** 

In [138]:
poets.image_url.nunique()

10020

In [139]:
poets.image_preview_url.nunique()

10020

In [141]:
poets.image_url.iloc[0]

'https://lh3.googleusercontent.com/ulT6Iu_7798lvdulT253Ubtgoi3JYYnn5fo_tQ4A5l2NFFIkLAfqjRm5o8W9UMffMg8a6CMDmU7IZ9rFnPbSPnZ7YPdKBM8AnWMJ'

In [140]:
poets.image_preview_url.iloc[0]

'https://lh3.googleusercontent.com/ulT6Iu_7798lvdulT253Ubtgoi3JYYnn5fo_tQ4A5l2NFFIkLAfqjRm5o8W9UMffMg8a6CMDmU7IZ9rFnPbSPnZ7YPdKBM8AnWMJ=s250'

In [142]:
poets.image_thumbnail_url.iloc[0]

'https://lh3.googleusercontent.com/ulT6Iu_7798lvdulT253Ubtgoi3JYYnn5fo_tQ4A5l2NFFIkLAfqjRm5o8W9UMffMg8a6CMDmU7IZ9rFnPbSPnZ7YPdKBM8AnWMJ=s128'

In [143]:
poets.image_original_url.iloc[0]

'https://d1xxei964ioe0z.cloudfront.net/full/a161f1a5f415f4e14c76105c15b190e15071c89be025845aad5bab3b25ebaccd.png'

Perfect, these are the columns we will use for scraping the images. We will keep the image_url & image_original_url column and decide which one we will use when we are scraping. 

**name** 

In [146]:
poets.name[:5]

0               I
1    Origin #7THK
2    Origin #6AF2
3            BOND
4    Origin #E0FR
Name: name, dtype: object

**asset_contract** 

In [148]:
poets.asset_contract[:5]

0    {'address': '0x4b3406a41399c7fd2ba65cbc93697ad...
1    {'address': '0x4b3406a41399c7fd2ba65cbc93697ad...
2    {'address': '0x4b3406a41399c7fd2ba65cbc93697ad...
3    {'address': '0x4b3406a41399c7fd2ba65cbc93697ad...
4    {'address': '0x4b3406a41399c7fd2ba65cbc93697ad...
Name: asset_contract, dtype: object

In [153]:
poets.asset_contract.iloc[100]

{'address': '0x4b3406a41399c7fd2ba65cbc93697ad9e7ea61e5',
 'asset_contract_type': 'non-fungible',
 'created_date': '2021-09-25T05:35:23.490231',
 'name': 'Lost Poets',
 'nft_version': '3.0',
 'opensea_version': None,
 'owner': 2504088,
 'schema_name': 'ERC721',
 'symbol': 'POETS',
 'total_supply': None,
 'description': '∴ AB AETERNO',
 'external_link': 'https://lostpoets.xyz/',
 'image_url': 'https://lh3.googleusercontent.com/cGKkMxKQ-7J0A4mZvcPqr2LKvGqIVsXjzGqpfxRNb4yVr-zTycPJ0kznkwKqDyUTg--A32oW8cI32BQBl6hrbM-frn1__o544ECpCg=s120',
 'default_to_fiat': False,
 'dev_buyer_fee_basis_points': 0,
 'dev_seller_fee_basis_points': 1000,
 'only_proxied_transfers': False,
 'opensea_buyer_fee_basis_points': 0,
 'opensea_seller_fee_basis_points': 250,
 'buyer_fee_basis_points': 0,
 'seller_fee_basis_points': 1250,
 'payout_address': '0x8e8717dcd1687080757cb513d7d6702625973be8'}

Valuable information but it is the same for each observation. Also we have this in our events/sales database; so not necessary to carry this info downstream. 

**permalink** 

In [154]:
poets.permalink[:5]

0    https://opensea.io/assets/0x4b3406a41399c7fd2b...
1    https://opensea.io/assets/0x4b3406a41399c7fd2b...
2    https://opensea.io/assets/0x4b3406a41399c7fd2b...
3    https://opensea.io/assets/0x4b3406a41399c7fd2b...
4    https://opensea.io/assets/0x4b3406a41399c7fd2b...
Name: permalink, dtype: object

In [158]:
poets.permalink.iloc[2]

'https://opensea.io/assets/0x4b3406a41399c7fd2ba65cbc93697ad9e7ea61e5/3'

This is the link that shows us the link in the Opensea website. Let's keep it with us for now. 

**collection** 

In [159]:
poets.collection[:5]

0    {'banner_image_url': 'https://lh3.googleuserco...
1    {'banner_image_url': 'https://lh3.googleuserco...
2    {'banner_image_url': 'https://lh3.googleuserco...
3    {'banner_image_url': 'https://lh3.googleuserco...
4    {'banner_image_url': 'https://lh3.googleuserco...
Name: collection, dtype: object

In [165]:
poets.collection.iloc[2]

{'banner_image_url': 'https://lh3.googleusercontent.com/VNQfgH6h0cuiP6U-tTZ8qRUVivEtB1TdMsn6MWG_15wzr1899ehqrVZ_vrh5a3uNkseRmTfLyi0X8Q-WOq06e3IhMSiCyHV6XX97oxM=s2500',
 'chat_url': None,
 'created_date': '2021-09-03T04:50:03.338901',
 'default_to_fiat': False,
 'description': '∴ AB AETERNO',
 'dev_buyer_fee_basis_points': '0',
 'dev_seller_fee_basis_points': '1000',
 'discord_url': None,
 'display_data': {'card_display_style': 'cover'},
 'external_url': 'https://lostpoets.xyz/',
 'featured': False,
 'featured_image_url': 'https://lh3.googleusercontent.com/VNQfgH6h0cuiP6U-tTZ8qRUVivEtB1TdMsn6MWG_15wzr1899ehqrVZ_vrh5a3uNkseRmTfLyi0X8Q-WOq06e3IhMSiCyHV6XX97oxM=s300',
 'hidden': False,
 'safelist_request_status': 'verified',
 'image_url': 'https://lh3.googleusercontent.com/cGKkMxKQ-7J0A4mZvcPqr2LKvGqIVsXjzGqpfxRNb4yVr-zTycPJ0kznkwKqDyUTg--A32oW8cI32BQBl6hrbM-frn1__o544ECpCg=s120',
 'is_subject_to_whitelist': False,
 'large_image_url': 'https://lh3.googleusercontent.com/VNQfgH6h0cuiP6U-tTZ8

This is general information about the collection. Great info in general, but since we are working on 1 collection only and hence the value is the same for each observation, we will not be using this data downstream. 

**decimals** 

In [167]:
poets.decimals[:5]

0    0
1    0
2    0
3    0
4    0
Name: decimals, dtype: int64

In [168]:
poets.decimals.nunique()

1

Another column that won't be needed downstream. 

**token_metadata** 

In [169]:
poets.token_metadata[:5]

0    https://lostpoets.api.manifoldxyz.dev/metadata/1
1    https://lostpoets.api.manifoldxyz.dev/metadata/2
2    https://lostpoets.api.manifoldxyz.dev/metadata/3
3    https://lostpoets.api.manifoldxyz.dev/metadata/4
4    https://lostpoets.api.manifoldxyz.dev/metadata/5
Name: token_metadata, dtype: object

In [171]:
poets.token_metadata.nunique()

10020

In [173]:
poets.token_metadata.iloc[0]

'https://lostpoets.api.manifoldxyz.dev/metadata/1'

The metadata has important information but we would need to do further scraping to get this info. We'll leave it behind for now but note its usefullness if we want to work with this data in the future. 

**owner** 

In [174]:
poets.owner[:5]

0    {'user': {'username': 'pak'}, 'profile_img_url...
1    {'user': {'username': None}, 'profile_img_url'...
2    {'user': {'username': 'LordTruffington_Vault'}...
3    {'user': {'username': 'TopHand'}, 'profile_img...
4    {'user': {'username': 'FVCKRENDER'}, 'profile_...
Name: owner, dtype: object

In [178]:
poets.owner.iloc[2]

{'user': {'username': 'LordTruffington_Vault'},
 'profile_img_url': 'https://storage.googleapis.com/opensea-static/opensea-profile/24.png',
 'address': '0x247c87f0705a82fc75811d04d01a5a22f736bbb3',
 'config': ''}

This is great data if we want to combine with the sales data. It shows the latest owner of each item as of now. Let's extract the info from the dictionary. 

**owner_username**

In [179]:
poets.owner.iloc[2]['user']['username']

'LordTruffington_Vault'

In [188]:
poets['owner_username_extra'] = (poets.owner.apply(lambda x:x['user']))
# poets['owner_username'] = (poets.owner.apply(lambda x: 'unknown'
#                               if (pd.isnull(x)) else x['symbol']))

In [189]:
poets.owner_username_extra.iloc[2]

{'username': 'LordTruffington_Vault'}

In [191]:
poets['owner_username'] = (poets.owner_username_extra.apply(lambda x:'unknown owner' 
                                                            if (pd.isnull(x)) 
                                                            else x['username']))

In [194]:
poets['owner_username'].value_counts()

nifty-gateway-omnibus    930
unknown owner            864
mexpex                   472
ELCAPITAN                119
Siblings                 102
                        ... 
cuicui                     1
_Ski                       1
shiomu                     1
taco                       1
oddref                     1
Name: owner_username, Length: 1978, dtype: int64

In [200]:
poets.drop(['owner_username_extra'], axis = 1, inplace = True)

**owner_address**

In [235]:
poets.owner.iloc[2]['address']

'0x247c87f0705a82fc75811d04d01a5a22f736bbb3'

In [236]:
poets['owner_address'] = (poets.owner.apply(lambda x:x['address']))

In [237]:
poets.owner_address.iloc[2]

'0x247c87f0705a82fc75811d04d01a5a22f736bbb3'

In [238]:
poets.owner_address.nunique()

2601

**sell_orders**

In [203]:
poets[poets.sell_orders.notnull() == True].head(2)

,_id,id,token_id,num_sales,image_url,image_preview_url,image_thumbnail_url,image_original_url,name,description,...,collection,decimals,token_metadata,owner,sell_orders,creator,traits,last_sale,is_presale,owner_username
11,61b6bdd216629c8ab9d260df,59351041,12,0,https://lh3.googleusercontent.com/2Eurj_xFsb12...,https://lh3.googleusercontent.com/2Eurj_xFsb12...,https://lh3.googleusercontent.com/2Eurj_xFsb12...,https://d1xxei964ioe0z.cloudfront.net/full/6ac...,LEGEND,No Description Yet,...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://lostpoets.api.manifoldxyz.dev/metadata/12,"{'user': {'username': 'snow_glacier'}, 'profil...",[{'created_date': '2021-09-25T19:17:23.314185'...,[{'created_date': '2021-09-25T19:17:23.314185'...,"[{'trait_type': 'Class', 'value': 'Origin', 'd...",NaN,False,snow_glacier
17,61b6bdd216629c8ab9d260e5,59351060,18,0,https://lh3.googleusercontent.com/LXrXttMs0rP8...,https://lh3.googleusercontent.com/LXrXttMs0rP8...,https://lh3.googleusercontent.com/LXrXttMs0rP8...,https://d1xxei964ioe0z.cloudfront.net/full/a32...,Origin #I5D1,No Description Yet,...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://lostpoets.api.manifoldxyz.dev/metadata/18,"{'user': {'username': 'ultraparadise'}, 'profi...",[{'created_date': '2021-10-09T19:48:12.127717'...,[{'created_date': '2021-10-09T19:48:12.127717'...,"[{'trait_type': 'Origin', 'value': 'I5D1', 'di...",NaN,False,ultraparadise


In [204]:
poets.sell_orders.iloc[11]

[{'created_date': '2021-09-25T19:17:23.314185',
  'closing_date': None,
  'closing_extendable': False,
  'expiration_time': 0,
  'listing_time': 1632597325,
  'order_hash': '0x27e5cec922e350cc73aa221929c49d5d7cdcfcbe9f903130363b7c88fe18b5c7',
  'metadata': {'asset': {'id': '12',
    'address': '0x4b3406a41399c7fd2ba65cbc93697ad9e7ea61e5'},
   'schema': 'ERC721'},
  'exchange': '0x7be8076f4ea4a4ad08075c2508e481d6c946d12b',
  'maker': {'user': 60074,
   'profile_img_url': 'https://storage.googleapis.com/opensea-static/opensea-profile/12.png',
   'address': '0xa362a9676db12606d339d10433c1af3618bca1d4',
   'config': ''},
  'taker': {'user': 1766,
   'profile_img_url': 'https://storage.googleapis.com/opensea-static/opensea-profile/1.png',
   'address': '0x0000000000000000000000000000000000000000',
   'config': ''},
  'current_price': '3690000000000000000000',
  'current_bounty': '36900000000000000000',
  'bounty_multiple': '0.01',
  'maker_relayer_fee': '1250',
  'taker_relayer_fee': '0',
 

Great info but mostly covered by our sales analysis so let's skip this as well. 

**traits**

In [222]:
poets.traits.iloc[0][5]['trait_type']

'Origin'

We have a lot of traits in there. Let's come back to this feature if we decide to use it as a feature downstream. 

**last_sale**

In [224]:
poets[poets.last_sale.notnull() == True]

,_id,id,token_id,num_sales,image_url,image_preview_url,image_thumbnail_url,image_original_url,name,description,...,collection,decimals,token_metadata,owner,sell_orders,creator,traits,last_sale,is_presale,owner_username
3,61b6bdd216629c8ab9d260d7,59351008,4,1,https://lh3.googleusercontent.com/KHWZ4GF4GOIU...,https://lh3.googleusercontent.com/KHWZ4GF4GOIU...,https://lh3.googleusercontent.com/KHWZ4GF4GOIU...,https://d1xxei964ioe0z.cloudfront.net/full/073...,BOND,No Description Yet,...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://lostpoets.api.manifoldxyz.dev/metadata/4,"{'user': {'username': 'TopHand'}, 'profile_img...",NaN,NaN,"[{'trait_type': 'Class', 'value': 'Origin', 'd...","{'asset': {'token_id': '4', 'decimals': 0}, 'a...",False,TopHand
30,61b6bdd216629c8ab9d260f2,59351097,31,1,https://lh3.googleusercontent.com/Tj44Eh9grNW-...,https://lh3.googleusercontent.com/Tj44Eh9grNW-...,https://lh3.googleusercontent.com/Tj44Eh9grNW-...,https://d1xxei964ioe0z.cloudfront.net/full/1e7...,Origin #8WI4,origin born into happiness\n\npatiently waitin...,...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://lostpoets.api.manifoldxyz.dev/metadata/31,"{'user': {'username': 'GAMT'}, 'profile_img_ur...",NaN,NaN,"[{'trait_type': 'Word', 'value': 'can', 'displ...","{'asset': {'token_id': '31', 'decimals': 0}, '...",False,GAMT
37,61b6bdd216629c8ab9d260f9,59351122,38,1,https://lh3.googleusercontent.com/SSiWWa0qau1V...,https://lh3.googleusercontent.com/SSiWWa0qau1V...,https://lh3.googleusercontent.com/SSiWWa0qau1V...,https://d1xxei964ioe0z.cloudfront.net/full/0c9...,Origin #G31Y,No Description Yet,...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://lostpoets.api.manifoldxyz.dev/metadata/38,"{'user': {'username': 'mexpex'}, 'profile_img_...",NaN,NaN,"[{'trait_type': 'Origin', 'value': 'G31Y', 'di...","{'asset': {'token_id': '38', 'decimals': 0}, '...",False,mexpex
38,61b6bdd216629c8ab9d260fa,59351130,39,1,https://lh3.googleusercontent.com/rtM20MYymWSG...,https://lh3.googleusercontent.com/rtM20MYymWSG...,https://lh3.googleusercontent.com/rtM20MYymWSG...,https://d1xxei964ioe0z.cloudfront.net/full/936...,Origin #VBIC,No Description Yet,...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://lostpoets.api.manifoldxyz.dev/metadata/39,"{'user': {'username': 'mexpex'}, 'profile_img_...",NaN,NaN,"[{'trait_type': 'Origin', 'value': 'VBIC', 'di...","{'asset': {'token_id': '39', 'decimals': 0}, '...",False,mexpex
44,61b6bdd316629c8ab9d26100,59351142,45,1,https://lh3.googleusercontent.com/6EdftsYnkULj...,https://lh3.googleusercontent.com/6EdftsYnkULj...,https://lh3.googleusercontent.com/6EdftsYnkULj...,https://d1xxei964ioe0z.cloudfront.net/full/2d2...,Origin #THRF,No Description Yet,...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://lostpoets.api.manifoldxyz.dev/metadata/45,"{'user': {'username': 'mexpex'}, 'profile_img_...",NaN,NaN,"[{'trait_type': 'Origin', 'value': 'THRF', 'di...","{'asset': {'token_id': '45', 'decimals': 0}, '...",False,mexpex
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10007,61b6bec716629c8ab9d287eb,59378572,10626,1,https://lh3.googleusercontent.com/YWOkRFcNQCor...,https://lh3.googleusercontent.com/YWOkRFcNQCor...,https://lh3.googleusercontent.com/YWOkRFcNQCor...,https://d1xxei964ioe0z.cloudfront.net/full/653...,Poet #10626,No Description Yet,...,{'banner_image_url': 'https://lh3.googleuserco...,0,https://lostpoets.api.manifoldxyz.dev/metadata...,{'user': {'username': 'nifty-gateway-omnibus'}...,NaN,NaN,"[{'trait_type': 'Rewrites', 'value': 0, 'displ...","{'asset': {'token_id': '10626', 'decimals': 0}...",False,nifty-gateway-omnibus
10011,61b6bec716629c8ab9d287ef,59378948,10630,2,https://lh3.googleusercontent.com/mu6SeyeccQtk...,https://lh3.googleusercontent.com/mu6SeyeccQtk...,https://lh3.googleusercontent.com/mu6SeyeccQtk...,https://d1xxei964ioe0z.cloudfront.net/full/179...,Poet #10630,No Description Yet,...,{'banner_image_url': 'https://l

In [225]:
poets.last_sale.iloc[3]

{'asset': {'token_id': '4', 'decimals': 0},
 'asset_bundle': None,
 'event_type': 'successful',
 'event_timestamp': '2021-09-26T23:27:53',
 'auction_type': None,
 'total_price': '37000000000000000000',
 'payment_token': {'id': 1,
  'symbol': 'ETH',
  'address': '0x0000000000000000000000000000000000000000',
  'image_url': 'https://storage.opensea.io/files/6f8e2979d428180222796ff4a33ab929.svg',
  'name': 'Ether',
  'decimals': 18,
  'eth_price': '1.000000000000000',
  'usd_price': '3979.219999999999800000'},
 'transaction': {'block_hash': '0x277517c12baa4d5889182fa726bad7345a72d4faa9eb7003cdd5d6cb8419fbf1',
  'block_number': '13304385',
  'from_account': {'user': {'username': 'TOPHANDS'},
   'profile_img_url': 'https://storage.googleapis.com/opensea-static/opensea-profile/32.png',
   'address': '0xfed751d3f6f122555c34c1726ff67eacdc500bfa',
   'config': ''},
  'id': 171368158,
  'timestamp': '2021-09-26T23:27:53',
  'to_account': {'user': {'username': 'OpenSea-Orders'},
   'profile_img_ur

Less than 20% have this value, so might also not carry downstream. 

**is_presale**

In [230]:
poets.is_presale.value_counts()

False    10020
Name: is_presale, dtype: int64

### 3. Save processed data

In [239]:
poets.columns

Index(['_id', 'id', 'token_id', 'num_sales', 'image_url', 'image_preview_url',
       'image_thumbnail_url', 'image_original_url', 'name', 'description',
       'asset_contract', 'permalink', 'collection', 'decimals',
       'token_metadata', 'owner', 'sell_orders', 'creator', 'traits',
       'last_sale', 'is_presale', 'owner_username', 'owner_address'],
      dtype='object')

Now let's pick the columns we cleaned and decided to take with us and save them into a new DF. 

In [240]:
poets_clean = poets[['token_id', 'num_sales', 'image_url',
                     'image_original_url', 'name', 'description',
                     'permalink', 'creator', 'traits',
                     'owner_username', 'owner_address']]

This is all great information. So let's pack all our data into a CSV file. 

In [241]:
poets_clean.to_csv('poets_clean.csv', index= False)